In [27]:
# Imports für Neo4j-Verbindung und Datenverarbeitung
from neo4j import GraphDatabase
import pandas as pd
import json
from typing import List, Dict, Any
import logging

# Logging konfigurieren
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class Neo4jConnector:
    def __init__(self, uri: str, user: str, password: str):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        
    def close(self):
        self.driver.close()
        
    def test_connection(self):
        """Testet die Verbindung zur Neo4j-Datenbank"""
        try:
            with self.driver.session() as session:
                result = session.run("RETURN 1 as test")
                return result.single()["test"] == 1
        except Exception as e:
            logger.error(f"Verbindungsfehler: {e}")
            return False

In [ ]:
# Neo4j-Verbindungsdaten (anpassen an deine lokale Konfiguration)
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "bundestag_password" 

# Verbindung erstellen und testen
connector = Neo4jConnector(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)

if connector.test_connection():
    print("✅ Verbindung zu Neo4j erfolgreich!")
else:
    print("❌ Verbindung zu Neo4j fehlgeschlagen!")

✅ Verbindung zu Neo4j erfolgreich!


In [35]:
def get_random_politician_with_content(self, limit: int = 1) -> List[Dict[str, Any]]:
    """Lädt einen zufälligen Politiker mit allen seinen Content-Nodes"""
    query = """
    MATCH (p:Politician)
    OPTIONAL MATCH (p)-[:HAS_CONTENT]->(c:Content)
    RETURN p, 
           collect(DISTINCT {
               id: c.id,
               section_header: c.section_header,
               section_content: c.section_content
           }) as contents
    ORDER BY rand()
    LIMIT $limit
    """
    
    try:
        with self.driver.session() as session:
            result = session.run(query, limit=limit)
            politicians = []
            for record in result:
                politician = dict(record["p"])
                politician["contents"] = record["contents"]
                politicians.append(politician)
            return politicians
    except Exception as e:
        logger.error(f"Fehler beim Laden der Politiker mit Content: {e}")
        return []
    
# Methode zur Klasse hinzufügen
Neo4jConnector.get_random_politician_with_content = get_random_politician_with_content

In [36]:
# Zufälligen Politiker laden
# Politiker mit Content laden
politician_with_content = connector.get_random_politician_with_content(limit=1)

if politician_with_content:
    politician = politician_with_content[0]
    print(f"🎯 Politiker: {politician.get('name', 'N/A')}")
    print(f"📊 Anzahl Content-Nodes: {len(politician['contents'])}")
    
    # Alle Content-Nodes durchgehen
    for i, content in enumerate(politician['contents']):
        print(f"\n📝 Content {i+1}:")
        print(f"  Header: {content.get('section_header', 'N/A')}")
        print(f"  Content: {content.get('section_content', 'N/A')[:150]}...")
        print(f"  ID: {content.get('id', 'N/A')}")
        
    # Politiker für weitere Verarbeitung speichern
    sample_politician_with_content = politician
else:
    print("❌ Kein Politiker gefunden!")

🎯 Politiker: N/A
📊 Anzahl Content-Nodes: 4

📝 Content 1:
  Header: Einzelnachweise
  Content: ↑ a b "In der großen Politik muss man dem Leben abschwören", 16. März 2019, Freie Presse ↑ a b Katharina Naumann: Die Geschichte meiner Familie. 2016....
  ID: 07e97aa97c3cd7bec465650c8e8dfe5acce55ddd

📝 Content 2:
  Header: Weblink
  Content: Helga Otto im Archiv der sozialen Demokratie...
  ID: e5f9bc3c8f7a76f8d95bad9bfa511b3855b20aeb

📝 Content 3:
  Header: Leben
  Content: Helga Otto wuchs in Bad Dürrenberg, Nerchau und Auschwitz auf. Im Zuge der Aktion Ossawakim – ihr Vater war eine der betroffenen Fachkräfte – wurde si...
  ID: f3b82cb04f61ffff27ea41a994e4e98ade611e32

📝 Content 4:
  Header: #
  Content: Helga Otto (* 28. November 1938 in Leipzig)[1] ist eine deutsche Politikerin (SPD)....
  ID: 1382e3c6e760a7ba66e1272c7ebf71736fe7b1d3


In [46]:
def export_all_politicians_with_content(self, output_file: str = "testdata/neo4j_data_politicians.json"):
    """Exportiert alle Politiker mit Content direkt aus Neo4j"""
    query = """
    MATCH (p:Politician)
    OPTIONAL MATCH (p)-[:HAS_CONTENT]->(c:Content)
    RETURN p, 
           collect(DISTINCT {
               content_id: c.id,
               section_header: c.section_header,
               section_content: c.section_content
           }) as neo4j_content
    """
    
    try:
        with self.driver.session() as session:
            result = session.run(query)
            politicians = []
            
            for record in result:
                # Alle Politiker-Eigenschaften übernehmen
                politician = dict(record["p"])
                
                # Neo4j interne Element-ID als eindeutige ID hinzufügen
                politician["neo4j_element_id"] = record["p"].element_id
                
                # Content hinzufügen
                politician["neo4j_content"] = record["neo4j_content"]
                politicians.append(politician)
            
            # JSON speichern
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(politicians, f, ensure_ascii=False, indent=2)
            
            print(f"✅ {len(politicians)} Politiker in {output_file} exportiert")
            return output_file
            
    except Exception as e:
        logger.error(f"Fehler beim Export: {e}")
        return None

# Methode zur Klasse hinzufügen
Neo4jConnector.export_all_politicians_with_content = export_all_politicians_with_content

In [47]:
# Alle Politiker mit Content exportieren
output_file = connector.export_all_politicians_with_content("testdata/neo4j_data_politicians.json")

if output_file:
    print(f"✅ Export erfolgreich: {output_file}")
    
    # Überprüfen der gespeicherten Datei
    import os
    if os.path.exists(output_file):
        file_size = os.path.getsize(output_file) / (1024 * 1024)  # MB
        print(f"�� Datei gespeichert: {file_size:.2f} MB")
    else:
        print("❌ Datei wurde nicht gefunden!")

    with open(output_file, 'r', encoding='utf-8') as f:
        politicians = json.load(f)
    
    if politicians:
        first_politician = politicians[0]
        print(f"🎯 Erster Politiker:")
        print(f"  Alle Eigenschaften: {list(first_politician.keys())}")
        print(f"  ID: {first_politician.get('id', 'N/A')}")
        print(f"  Name: {first_politician.get('full_name', 'N/A')}")
        print(f"  Content-Nodes: {len(first_politician.get('neo4j_content', []))}")
else:
    print("❌ Export fehlgeschlagen!")

✅ 4465 Politiker in testdata/neo4j_data_politicians.json exportiert
✅ Export erfolgreich: testdata/neo4j_data_politicians.json
�� Datei gespeichert: 35.42 MB
🎯 Erster Politiker:
  Alle Eigenschaften: ['detail_page', 'firstname', 'full_name', 'death_year', 'birth_year', 'lastname', 'neo4j_element_id', 'neo4j_content']
  ID: N/A
  Name: Gerhart Baum
  Content-Nodes: 10


In [9]:
# Neo4j-Verbindung sauber schließen
try:
    connector.close()
    print("✅ Neo4j-Verbindung erfolgreich geschlossen")
except Exception as e:
    print(f"⚠️ Fehler beim Schließen der Verbindung: {e}")
finally:
    print("🔒 Cleanup abgeschlossen")

✅ Neo4j-Verbindung erfolgreich geschlossen
🔒 Cleanup abgeschlossen
